# 02 - Feature Engineering

This notebook performs feature engineering for the maternal health risk prediction model.

## Objectives
1. Handle missing values with appropriate imputation strategies
2. Encode categorical variables for machine learning
3. Create derived features (interactions, bins, polynomial)
4. Build feature sets for model training
5. Save processed data for modeling

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import (
    StandardScaler, MinMaxScaler, RobustScaler,
    OneHotEncoder, OrdinalEncoder, LabelEncoder
)
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

# Import our custom modules
import sys
sys.path.append('..')
from src.data_loader import (
    create_high_risk_indicator,
    create_demographic_groups
)

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
%matplotlib inline

print("Libraries loaded successfully")

## 1. Load Data

We recreate the synthetic demonstration dataset from notebook 01. In production, load the actual DHS data.

In [ ]:
# For demonstration, create synthetic data matching DHS structure
# In production: df = pd.read_csv('../data/processed/maternal_health_combined.csv')

print("Creating synthetic demonstration data...")
print("(Replace with actual DHS data loading when files are available)\n")

np.random.seed(42)
n_samples = 5000

# Base features
df = pd.DataFrame({
    'country': np.random.choice(['NG', 'KE', 'GH', 'UG', 'TZ'], n_samples, p=[0.35, 0.2, 0.15, 0.15, 0.15]),
    'age': np.random.normal(28, 7, n_samples).clip(15, 49).astype(int),
    'residence_type': np.random.choice([1, 2], n_samples, p=[0.35, 0.65]),
    'education_level': np.random.choice([0, 1, 2, 3], n_samples, p=[0.15, 0.35, 0.35, 0.15]),
    'wealth_index': np.random.choice([1, 2, 3, 4, 5], n_samples),
    'total_children_born': np.random.poisson(2.5, n_samples),
    'anemia_level': np.random.choice([1, 2, 3, 4], n_samples, p=[0.05, 0.15, 0.25, 0.55]),
    'birth_interval_months': np.random.exponential(30, n_samples).astype(int),
    'antenatal_visits': np.random.poisson(4, n_samples).clip(0, 15),
    'bmi': np.random.normal(24, 5, n_samples).clip(15, 45),
    'distance_health_facility_problem': np.random.choice([0, 1], n_samples, p=[0.6, 0.4]),
    'health_insurance': np.random.choice([0, 1], n_samples, p=[0.85, 0.15]),
    'pregnancy_terminated': np.random.choice([0, 1], n_samples, p=[0.85, 0.15]),
    'region': np.random.randint(1, 8, n_samples),
})

# Introduce realistic missing values
missing_rates = {
    'birth_interval_months': 0.15,  # Missing for first births
    'anemia_level': 0.10,           # Not all tested
    'bmi': 0.08,                    # Not all measured
    'antenatal_visits': 0.05,       # Some missing
    'health_insurance': 0.03,       # Some unknown
}

for col, rate in missing_rates.items():
    missing_mask = np.random.random(n_samples) < rate
    df.loc[missing_mask, col] = np.nan

print(f"Dataset shape: {df.shape}")
df.head()

## 2. Assess Missing Values

Understanding missing data patterns is critical for choosing appropriate imputation strategies.

In [ ]:
# Calculate missing value statistics
missing_stats = pd.DataFrame({
    'Missing Count': df.isnull().sum(),
    'Missing %': (df.isnull().sum() / len(df) * 100).round(2),
    'Data Type': df.dtypes
})
missing_stats = missing_stats[missing_stats['Missing Count'] > 0].sort_values('Missing %', ascending=False)

print("Missing Value Summary:")
print("=" * 50)
print(missing_stats)
print(f"\nTotal rows: {len(df):,}")
print(f"Complete cases: {df.dropna().shape[0]:,} ({df.dropna().shape[0]/len(df)*100:.1f}%)")

In [ ]:
# Visualize missing data patterns
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Missing value counts
if len(missing_stats) > 0:
    missing_stats['Missing %'].plot(kind='barh', ax=axes[0], color='coral')
    axes[0].set_xlabel('Missing Percentage')
    axes[0].set_title('Missing Values by Feature')
    axes[0].axvline(x=5, color='red', linestyle='--', alpha=0.7, label='5% threshold')
    axes[0].legend()
else:
    axes[0].text(0.5, 0.5, 'No missing values', ha='center', va='center', fontsize=14)
    axes[0].set_title('Missing Values by Feature')

# Missing data heatmap (sample)
sample_df = df.sample(min(100, len(df)), random_state=42)
sns.heatmap(sample_df.isnull(), cbar=True, yticklabels=False, ax=axes[1], cmap='viridis')
axes[1].set_title('Missing Data Pattern (Sample of 100 rows)')

plt.tight_layout()
plt.savefig('../results/figures/missing_values_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Handle Missing Values

We apply different imputation strategies based on variable type and missingness pattern:

| Strategy | Use Case |
|----------|----------|
| Median | Skewed numeric variables |
| Mean | Normally distributed numeric variables |
| Mode | Categorical variables |
| KNN | Complex patterns with multiple missing |
| Indicator | When missingness is informative |

In [ ]:
# Create a copy for imputation
df_imputed = df.copy()

# Define imputation strategies by variable
imputation_strategies = {
    # Numeric - use median for robustness to outliers
    'birth_interval_months': 'median',
    'bmi': 'median',
    'antenatal_visits': 'median',
    
    # Categorical/ordinal - use mode
    'anemia_level': 'mode',
    'health_insurance': 'mode',
}

# Create missing indicators for variables where missingness may be informative
informative_missing = ['birth_interval_months', 'anemia_level']

for col in informative_missing:
    if col in df_imputed.columns:
        df_imputed[f'{col}_missing'] = df_imputed[col].isnull().astype(int)

print("Created missing indicators for:")
print(informative_missing)

In [ ]:
# Apply imputation
for col, strategy in imputation_strategies.items():
    if col not in df_imputed.columns:
        continue
        
    missing_count = df_imputed[col].isnull().sum()
    if missing_count == 0:
        continue
    
    if strategy == 'median':
        fill_value = df_imputed[col].median()
    elif strategy == 'mean':
        fill_value = df_imputed[col].mean()
    elif strategy == 'mode':
        fill_value = df_imputed[col].mode()[0]
    else:
        fill_value = strategy  # Use provided value
    
    df_imputed[col] = df_imputed[col].fillna(fill_value)
    print(f"{col}: imputed {missing_count} values with {strategy} ({fill_value:.2f})")

# Verify no missing values remain in key columns
print(f"\nRemaining missing values: {df_imputed[list(imputation_strategies.keys())].isnull().sum().sum()}")

## 4. Encode Categorical Variables

We use different encoding strategies based on variable type:

- **Ordinal Encoding**: For ordered categories (education level, wealth index)
- **One-Hot Encoding**: For nominal categories (country, residence)
- **Binary Encoding**: For binary variables

In [ ]:
# Define variable types
ordinal_vars = {
    'education_level': [0, 1, 2, 3],  # None < Primary < Secondary < Higher
    'wealth_index': [1, 2, 3, 4, 5],   # Poorest to Richest
    'anemia_level': [1, 2, 3, 4],      # Severe < Moderate < Mild < Not anemic
}

nominal_vars = ['country']  # No natural ordering

binary_vars = ['residence_type', 'distance_health_facility_problem', 
               'health_insurance', 'pregnancy_terminated']

print("Variable categorization:")
print(f"  Ordinal: {list(ordinal_vars.keys())}")
print(f"  Nominal: {nominal_vars}")
print(f"  Binary: {binary_vars}")

In [ ]:
# One-Hot Encoding for nominal variables
for col in nominal_vars:
    if col in df_imputed.columns:
        dummies = pd.get_dummies(df_imputed[col], prefix=col, drop_first=False)
        df_imputed = pd.concat([df_imputed, dummies], axis=1)
        print(f"One-hot encoded {col}: created {len(dummies.columns)} columns")
        print(f"  Columns: {list(dummies.columns)}")

print(f"\nDataset shape after one-hot encoding: {df_imputed.shape}")

In [ ]:
# Create readable labels for demographic analysis
df_imputed = create_demographic_groups(df_imputed)

# Create binary encodings (already numeric but ensure proper coding)
binary_mappings = {
    'residence_type': {1: 0, 2: 1},  # 0=Urban, 1=Rural
}

for col, mapping in binary_mappings.items():
    if col in df_imputed.columns:
        df_imputed[f'{col}_encoded'] = df_imputed[col].map(mapping)
        print(f"Created {col}_encoded with mapping {mapping}")

## 5. Create Derived Features

Engineering new features that may capture important risk patterns.

In [ ]:
# Age-related features
df_imputed['age_squared'] = df_imputed['age'] ** 2
df_imputed['age_risk_young'] = (df_imputed['age'] < 18).astype(int)
df_imputed['age_risk_old'] = (df_imputed['age'] > 35).astype(int)
df_imputed['age_risk_any'] = (df_imputed['age_risk_young'] | df_imputed['age_risk_old']).astype(int)

# Age groups for analysis
df_imputed['age_group'] = pd.cut(
    df_imputed['age'], 
    bins=[14, 19, 24, 29, 34, 39, 50],
    labels=['15-19', '20-24', '25-29', '30-34', '35-39', '40+']
)

print("Age-derived features created:")
print("  - age_squared: polynomial feature")
print("  - age_risk_young: <18 years indicator")
print("  - age_risk_old: >35 years indicator")
print("  - age_risk_any: combined age risk")
print("  - age_group: categorical age bands")

In [ ]:
# Parity and birth interval features
df_imputed['high_parity'] = (df_imputed['total_children_born'] > 4).astype(int)
df_imputed['first_birth'] = (df_imputed['total_children_born'] == 0).astype(int)

# Short birth interval (< 24 months)
df_imputed['short_birth_interval'] = (df_imputed['birth_interval_months'] < 24).astype(int)

# Parity groups
df_imputed['parity_group'] = pd.cut(
    df_imputed['total_children_born'],
    bins=[-1, 0, 2, 4, 100],
    labels=['0', '1-2', '3-4', '5+']
)

print("\nParity features created:")
print("  - high_parity: >4 children indicator")
print("  - first_birth: first pregnancy indicator")
print("  - short_birth_interval: <24 months indicator")
print("  - parity_group: categorical parity bands")

In [ ]:
# Health indicator features
df_imputed['anemia_severe_moderate'] = df_imputed['anemia_level'].isin([1, 2]).astype(int)
df_imputed['anemia_any'] = (df_imputed['anemia_level'] < 4).astype(int)

# BMI categories (WHO classification)
df_imputed['bmi_underweight'] = (df_imputed['bmi'] < 18.5).astype(int)
df_imputed['bmi_overweight'] = (df_imputed['bmi'] >= 25).astype(int)
df_imputed['bmi_obese'] = (df_imputed['bmi'] >= 30).astype(int)
df_imputed['bmi_risk'] = (df_imputed['bmi_underweight'] | df_imputed['bmi_obese']).astype(int)

df_imputed['bmi_category'] = pd.cut(
    df_imputed['bmi'],
    bins=[0, 18.5, 25, 30, 100],
    labels=['Underweight', 'Normal', 'Overweight', 'Obese']
)

print("\nHealth indicator features created:")
print("  - anemia_severe_moderate: high-risk anemia")
print("  - anemia_any: any anemia present")
print("  - bmi_underweight, bmi_overweight, bmi_obese: BMI risk indicators")
print("  - bmi_risk: combined BMI risk (underweight or obese)")
print("  - bmi_category: WHO BMI classification")

In [ ]:
# Healthcare access features
df_imputed['adequate_anc'] = (df_imputed['antenatal_visits'] >= 4).astype(int)  # WHO recommended minimum
df_imputed['no_anc'] = (df_imputed['antenatal_visits'] == 0).astype(int)

# Healthcare access composite
df_imputed['healthcare_barriers'] = (
    df_imputed['distance_health_facility_problem'] + 
    (1 - df_imputed['health_insurance'])
).clip(0, 2)

print("\nHealthcare access features created:")
print("  - adequate_anc: >=4 ANC visits (WHO standard)")
print("  - no_anc: zero ANC visits")
print("  - healthcare_barriers: composite access barrier score (0-2)")

In [ ]:
# Interaction features (socioeconomic with health)
df_imputed['poor_rural'] = (
    (df_imputed['wealth_index'] <= 2) & 
    (df_imputed['residence_type'] == 2)
).astype(int)

df_imputed['poor_uneducated'] = (
    (df_imputed['wealth_index'] <= 2) & 
    (df_imputed['education_level'] <= 1)
).astype(int)

df_imputed['rural_no_insurance'] = (
    (df_imputed['residence_type'] == 2) & 
    (df_imputed['health_insurance'] == 0)
).astype(int)

df_imputed['young_high_parity'] = (
    (df_imputed['age'] < 25) & 
    (df_imputed['total_children_born'] >= 3)
).astype(int)

# Cumulative vulnerability score
df_imputed['vulnerability_score'] = (
    (df_imputed['wealth_index'] <= 2).astype(int) +
    (df_imputed['education_level'] <= 1).astype(int) +
    (df_imputed['residence_type'] == 2).astype(int) +
    df_imputed['distance_health_facility_problem'] +
    (1 - df_imputed['health_insurance'])
).astype(int)

print("\nInteraction features created:")
print("  - poor_rural: low wealth AND rural residence")
print("  - poor_uneducated: low wealth AND low education")
print("  - rural_no_insurance: rural AND uninsured")
print("  - young_high_parity: young mother with many children")
print("  - vulnerability_score: cumulative socioeconomic risk (0-5)")

## 6. Create Target Variable

In [ ]:
# Create the high-risk pregnancy indicator
df_imputed['high_risk'] = create_high_risk_indicator(df_imputed)

print("Target variable distribution:")
print(df_imputed['high_risk'].value_counts(normalize=True).round(3))
print(f"\nClass balance ratio: {df_imputed['high_risk'].mean():.2%} high-risk")

## 7. Define Feature Sets

Create organized feature sets for different modeling approaches.

In [ ]:
# Define feature sets
FEATURE_SETS = {
    # Core clinical features (minimal)
    'clinical_minimal': [
        'age', 'total_children_born', 'birth_interval_months',
        'anemia_level', 'bmi', 'pregnancy_terminated'
    ],
    
    # Extended clinical features
    'clinical_extended': [
        'age', 'age_squared', 'total_children_born', 'birth_interval_months',
        'anemia_level', 'bmi', 'pregnancy_terminated', 'antenatal_visits',
        'age_risk_any', 'high_parity', 'short_birth_interval',
        'anemia_severe_moderate', 'bmi_risk', 'adequate_anc'
    ],
    
    # Socioeconomic features
    'socioeconomic': [
        'wealth_index', 'education_level', 'residence_type_encoded',
        'distance_health_facility_problem', 'health_insurance',
        'healthcare_barriers', 'vulnerability_score'
    ],
    
    # Full feature set (clinical + socioeconomic)
    'full': [
        # Demographics
        'age', 'age_squared', 'residence_type_encoded',
        'education_level', 'wealth_index',
        
        # Clinical
        'total_children_born', 'birth_interval_months',
        'anemia_level', 'bmi', 'pregnancy_terminated',
        'antenatal_visits',
        
        # Healthcare access
        'distance_health_facility_problem', 'health_insurance',
        
        # Derived clinical
        'age_risk_any', 'high_parity', 'short_birth_interval',
        'anemia_severe_moderate', 'bmi_risk', 'adequate_anc',
        
        # Derived socioeconomic
        'healthcare_barriers', 'vulnerability_score',
        
        # Interactions
        'poor_rural', 'poor_uneducated', 'rural_no_insurance',
        
        # Missing indicators
        'birth_interval_months_missing', 'anemia_level_missing'
    ],
    
    # Features with country dummies (for pooled models)
    'full_with_country': [
        # All full features plus country indicators
        'age', 'age_squared', 'residence_type_encoded',
        'education_level', 'wealth_index',
        'total_children_born', 'birth_interval_months',
        'anemia_level', 'bmi', 'pregnancy_terminated',
        'antenatal_visits',
        'distance_health_facility_problem', 'health_insurance',
        'age_risk_any', 'high_parity', 'short_birth_interval',
        'anemia_severe_moderate', 'bmi_risk', 'adequate_anc',
        'healthcare_barriers', 'vulnerability_score',
        'poor_rural', 'poor_uneducated', 'rural_no_insurance',
        'birth_interval_months_missing', 'anemia_level_missing',
        # Country dummies
        'country_GH', 'country_KE', 'country_NG', 'country_TZ', 'country_UG'
    ]
}

# Sensitive attributes for fairness analysis
SENSITIVE_ATTRIBUTES = [
    'wealth_group',      # 5 categories
    'wealth_binary',     # Poor vs Non-poor
    'residence',         # Urban vs Rural
    'education_group',   # 4 categories
    'education_binary',  # Secondary+ vs Below
    'country'            # 5 countries
]

print("Feature sets defined:")
for name, features in FEATURE_SETS.items():
    print(f"  {name}: {len(features)} features")

In [ ]:
# Verify all features exist in dataframe
print("Verifying feature availability:")
print("=" * 50)

for set_name, features in FEATURE_SETS.items():
    available = [f for f in features if f in df_imputed.columns]
    missing = [f for f in features if f not in df_imputed.columns]
    
    status = "OK" if len(missing) == 0 else "MISSING"
    print(f"\n{set_name}: {status}")
    print(f"  Available: {len(available)}/{len(features)}")
    if missing:
        print(f"  Missing: {missing}")

## 8. Feature Scaling Preparation

Prepare scalers for numeric features. Note: Scaling should be fit on training data only to prevent data leakage.

In [ ]:
# Define which features need scaling
numeric_features_to_scale = [
    'age', 'age_squared', 'total_children_born', 'birth_interval_months',
    'bmi', 'antenatal_visits', 'vulnerability_score', 'healthcare_barriers'
]

# Features that are already on appropriate scales (binary, ordinal 0-5, etc.)
no_scaling_needed = [
    'wealth_index', 'education_level', 'anemia_level', 'residence_type_encoded',
    'distance_health_facility_problem', 'health_insurance', 'pregnancy_terminated',
    'age_risk_any', 'high_parity', 'short_birth_interval', 'anemia_severe_moderate',
    'bmi_risk', 'adequate_anc', 'poor_rural', 'poor_uneducated', 'rural_no_insurance',
    'birth_interval_months_missing', 'anemia_level_missing'
]

print("Scaling plan:")
print(f"  To scale: {len(numeric_features_to_scale)} features")
print(f"  No scaling: {len(no_scaling_needed)} features")

In [ ]:
# Examine distributions of numeric features
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, col in enumerate(numeric_features_to_scale):
    if col in df_imputed.columns and i < len(axes):
        df_imputed[col].hist(bins=30, ax=axes[i], color='steelblue', edgecolor='white')
        axes[i].set_title(col)
        axes[i].set_xlabel('')

# Hide unused subplots
for j in range(i+1, len(axes)):
    axes[j].set_visible(False)

plt.suptitle('Distribution of Features to Scale', y=1.02, fontsize=14)
plt.tight_layout()
plt.savefig('../results/figures/feature_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Feature Correlation Analysis

In [ ]:
# Calculate correlations for full feature set
full_features = [f for f in FEATURE_SETS['full'] if f in df_imputed.columns]
corr_matrix = df_imputed[full_features + ['high_risk']].corr()

# Plot correlation with target
target_corr = corr_matrix['high_risk'].drop('high_risk').sort_values(key=abs, ascending=True)

plt.figure(figsize=(10, 12))
colors = ['indianred' if x > 0 else 'steelblue' for x in target_corr]
target_corr.plot(kind='barh', color=colors)
plt.xlabel('Correlation with High Risk')
plt.title('Feature Correlation with Target Variable')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.savefig('../results/figures/feature_target_correlation.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Check for highly correlated features (potential multicollinearity)
high_corr_threshold = 0.8

# Get upper triangle of correlation matrix
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find highly correlated pairs
high_corr_pairs = []
for col in upper_tri.columns:
    for idx in upper_tri.index:
        if abs(upper_tri.loc[idx, col]) > high_corr_threshold:
            high_corr_pairs.append((idx, col, upper_tri.loc[idx, col]))

print(f"Highly correlated feature pairs (|r| > {high_corr_threshold}):")
print("=" * 60)
if high_corr_pairs:
    for f1, f2, corr in sorted(high_corr_pairs, key=lambda x: abs(x[2]), reverse=True):
        print(f"  {f1} <-> {f2}: {corr:.3f}")
else:
    print("  No highly correlated pairs found")

## 10. Summary and Data Export

In [ ]:
# Final dataset summary
print("FEATURE ENGINEERING SUMMARY")
print("=" * 60)
print(f"\nFinal dataset shape: {df_imputed.shape}")
print(f"Total samples: {len(df_imputed):,}")
print(f"Total features: {len(df_imputed.columns)}")

print(f"\nTarget distribution:")
print(f"  High-risk: {df_imputed['high_risk'].sum():,} ({df_imputed['high_risk'].mean():.1%})")
print(f"  Low-risk: {(1-df_imputed['high_risk']).sum():,} ({(1-df_imputed['high_risk'].mean()):.1%})")

print(f"\nMissing values remaining: {df_imputed.isnull().sum().sum()}")

print(f"\nFeature sets available:")
for name, features in FEATURE_SETS.items():
    available = sum(1 for f in features if f in df_imputed.columns)
    print(f"  {name}: {available} features")

In [ ]:
# Save processed dataset
output_path = '../data/processed/maternal_health_features.csv'
df_imputed.to_csv(output_path, index=False)
print(f"Saved processed data to: {output_path}")

# Save feature set definitions
import json
feature_sets_path = '../data/processed/feature_sets.json'
with open(feature_sets_path, 'w') as f:
    json.dump({
        'feature_sets': FEATURE_SETS,
        'sensitive_attributes': SENSITIVE_ATTRIBUTES,
        'numeric_features_to_scale': numeric_features_to_scale,
        'target_variable': 'high_risk'
    }, f, indent=2)
print(f"Saved feature definitions to: {feature_sets_path}")

In [ ]:
print("\nFeature engineering complete!")
print("Next: 03_model_training.ipynb")